<img src="https://hilpisch.com/tpq_logo_bic.png" width="20%" align="right">


# Python & AI in Asset Management
## Covariance Matrix, Eigenvalues & Eigenvectors

&copy; Dr. Yves J. Hilpisch<br>
AI-Powered by GPT 5.x<br>
The Python Quants GmbH | https://tpq.io<br>
https://hilpisch.com | https://linktr.ee/dyjh


## Notebook Goals

This notebook is a deep, end-to-end case study of the **2×2 covariance matrix**

$$
\Sigma=\begin{pmatrix}
\sigma_1^2 & \sigma_{12}\\
\sigma_{12} & \sigma_2^2
\end{pmatrix},
$$

You will learn to:

- move from **state-contingent payoffs** to expectations, variances, covariance, and correlation,
- read the **shape of the payoff cloud** in the plane and connect it to the covariance matrix,
- interpret **eigenvalues and eigenvectors** as risk magnitudes and principal directions, and
- relate these ideas back to **two-asset portfolios**, principal components, and near-singular covariance matrices.

The focus is on **geometry and intuition**: the formulas are short, but every key object has a picture and an interpretation.

All plots use **Matplotlib** only.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# plt.style.use('dark_background') # e.g. for notebook only
plt.style.use('seaborn-v0_8')

np.set_printoptions(precision=6, suppress=True)
%config InlineBackend.figure_format = 'svg'

def cov_from_states(x, p):
    """Probability-weighted covariance matrix for discrete states.
    
    Parameters
    ----------
    x : array, shape (n_states, n_assets)
        State-contingent payoffs/returns (rows=states, cols=assets).
    p : array, shape (n_states,)
        Probabilities (need not sum to 1; will be normalized).

    Returns
    -------
    mu : array, shape (n_assets,)
        Mean vector E[X]
    Sigma : array, shape (n_assets, n_assets)
        Covariance matrix E[(X-mu)(X-mu)^T]
    """
    p = np.asarray(p, dtype=float)
    p = p / p.sum()
    x = np.asarray(x, dtype=float)
    mu = (p[:, None] * x).sum(axis=0)
    xc = x - mu
    Sigma = xc.T @ (p[:, None] * xc)
    return mu, Sigma

def corr_from_cov(S):
    d = np.sqrt(np.diag(S))
    return S / np.outer(d, d)

def eig_sorted(S):
    # For symmetric matrices: eigh returns real eigenvalues + orthonormal eigenvectors.
    vals, vecs = np.linalg.eigh(S)
    idx = np.argsort(vals)[::-1]
    return vals[idx], vecs[:, idx]

def unit(v):
    v = np.asarray(v, float)
    n = np.linalg.norm(v)
    return v if n == 0 else v/n

from pathlib import Path

def fig_dir():
    cwd = Path.cwd()  # current working directory
    return cwd / "figures"

FIG_DIR = fig_dir()
FIG_DIR.mkdir(parents=True, exist_ok=True)

def save_fig(fig, name):
    fig.savefig(
        FIG_DIR / name, dpi=300, bbox_inches="tight"
    )  # save figure to figures/


## 1) Start from primitives: a static economy with discrete states

We consider a one-period model with a finite state space $\Omega=\{\omega_1,\dots,\omega_N\}$ and probabilities $p(\omega)$.
Two assets produce (random) payoffs $X_1(\omega)$ and $X_2(\omega)$. A single outcome $\omega_k$ therefore lives as a point in the plane.

From these primitives we derive the familiar summary statistics:

- expectations $\mu_i=\mathbb{E}[X_i]$ (average payoff),
- variances $\sigma_i^2=\mathbb{V}[X_i]$ (spread of each payoff around its mean),
- covariance $\sigma_{12}=\mathrm{Cov}(X_1,X_2)$ (do payoffs move together or offset each other?),
- correlation $\rho_{12}=\sigma_{12}/(\sigma_1\sigma_2)$ (a scale-free version of covariance), and
- the covariance matrix $\Sigma$ collecting all pairwise covariances.

You can picture $X_1$ and $X_2$ as **two coordinates of the same economic world** (for example, payoffs of two assets in the same scenario set). Covariance will tell us whether that world tilts along the $(1,1)$ direction (strong positive co-movement), the $(1,-1)$ direction (hedging), or something in between.


In [ ]:
# Example economy: 6 discrete states
states = np.array(["Boom", "Good", "Normal", "Soft", "Recession", "Crisis"])
p = np.array([0.10, 0.20, 0.30, 0.20, 0.15, 0.05])  # probabilities

# Two assets: state-contingent payoffs (could be returns or cashflows)
X1 = np.array([1.40, 1.20, 1.05, 0.95, 0.80, 0.60])
X2 = np.array([1.15, 1.10, 1.02, 0.98, 0.90, 0.75])

X = np.column_stack([X1, X2])

mu, Sigma = cov_from_states(X, p)
rho = corr_from_cov(Sigma)

print("mu =", mu)
print("Sigma =\n", Sigma)
print("rho =\n", rho)


### 1.1 Visualize the state cloud in payoff space

Each state $\omega_k$ is a point $(X_1(\omega_k),X_2(\omega_k))$ in $\mathbb{R}^2$.

Mentally, think of **throwing one dart per state** at the payoff plane: dense regions of darts show typical joint outcomes, while extreme darts correspond to tail states. The rest of the notebook is about reading structure from this cloud.


In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(X1, X2)

for i, s in enumerate(states):
    ax.annotate(s, (X1[i], X2[i]), textcoords="offset points", xytext=(6, 6))

ax.set_xlabel("Asset 1 payoff $X_1$")
ax.set_ylabel("Asset 2 payoff $X_2$")
ax.set_title("Discrete states as a cloud in payoff space")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_state_cloud.png")
plt.show()


### 1.2 Centered payoffs and covariance as a weighted inner product

Let $\mu=\mathbb{E}[X]$ and $X^c(\omega)=X(\omega)-\mu$. Then:

$$
\Sigma=\mathbb{E}\left[(X-\mu)(X-\mu)^\top\right].
$$

In components:

$$
\sigma_{12} = \sum_{k=1}^N p_k \,(X_1(\omega_k)-\mu_1)(X_2(\omega_k)-\mu_2).
$$

So covariance is a **weighted inner product of centered payoffs**: we multiply deviations of $X_1$ and $X_2$ in each state and average them with state probabilities.

- **Sign:** positive if deviations tend to align (large with large, small with small); negative if they offset (one large when the other is small).
- **Units:** covariance has units of $X_1\cdot X_2$, so it depends on scaling; correlation is unitless and lives in $[-1,1]$.

A useful picture is to imagine **two centered time series as arrows** in a high-dimensional space (one coordinate per state or date). Covariance then measures the cosine of the angle between these arrows, scaled by their lengths.


In [ ]:
Xc = X - mu

sigma12 = Sigma[0, 1]
sigma1 = np.sqrt(Sigma[0, 0])
sigma2 = np.sqrt(Sigma[1, 1])
rho12 = sigma12 / (sigma1 * sigma2)

print("Centered states Xc (rows=states, cols=assets):\n", Xc)
print(f"\nσ1 = {sigma1:.6f}  σ2 = {sigma2:.6f}")
print(f"σ12 = {sigma12:.6f}  ρ12 = {rho12:.6f}")


## 2) Geometry: covariance ellipse and principal axes

For a symmetric covariance matrix, eigenvectors are orthonormal directions $u_1,u_2$ and eigenvalues $\lambda_1\ge\lambda_2\ge 0$ satisfy:

$$
\Sigma u_i = \lambda_i u_i,\quad u_i^\top u_j=\delta_{ij}.
$$

Key fact (unit vector $u$):

$$
\mathbb{V}[u^\top X] = u^\top \Sigma u.
$$

If $u$ is an eigenvector, then $u^\top \Sigma u = \lambda$.  So eigenvalues are **variances along eigenvector directions**.

To visualize this, we draw the **1-sigma ellipse** of the centered payoff cloud. For a bivariate normal distribution, the 1-sigma ellipse contains roughly $68\%$ of the probability mass, just like the familiar $\pm 1$ standard deviation interval on the real line. In our finite-state picture, it behaves like a **rubber band** wrapped around the most typical states, aligned with the directions of greatest and smallest variability.

Formally, we map the unit circle through $\sqrt{\Sigma}$: every point on the circle is stretched more along high-variance eigenvector directions and less along low-variance ones. The result is an ellipse whose axes line up with eigenvectors and whose axis lengths are proportional to $\sqrt{\lambda_i}$.


In [ ]:
vals, vecs = eig_sorted(Sigma)

# sqrt(Sigma) from eigen-decomposition: Sigma = Q Λ Q^T  => sqrt(Sigma)=Q sqrt(Λ) Q^T
sqrtSigma = vecs @ np.diag(np.sqrt(np.maximum(vals, 0))) @ vecs.T

theta = np.linspace(0, 2*np.pi, 400)
circle = np.column_stack([np.cos(theta), np.sin(theta)])
ellipse = circle @ sqrtSigma.T

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(Xc[:, 0], Xc[:, 1], label="Centered states")
ax.plot(ellipse[:, 0], ellipse[:, 1], linewidth=2, label="1-sigma ellipse")

ax.axhline(0, linewidth=1)
ax.axvline(0, linewidth=1)

ax.set_xlabel("Centered payoff $X_1-\\mu_1$")
ax.set_ylabel("Centered payoff $X_2-\\mu_2$")
ax.set_title("Centered state cloud + covariance ellipse")
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(fig, "fig_cov_ellipse.png")
plt.show()

print("Eigenvalues (descending):", vals)
print("Eigenvectors (columns):\n", vecs)


### 2.1 Draw eigenvectors on the ellipse

Eigenvectors are the **principal axes** of the ellipse; $\sqrt{\lambda_i}$ are the axis lengths (for the 1-sigma ellipse).

You can think of $u_1$ as the direction of **maximum spread** of the payoff cloud: if you project all centered states onto $u_1$, you see the largest variance. The second eigenvector $u_2$ is orthogonal to $u_1$ and captures the remaining variation. In factor-language, $u_1$ is the dominant “factor portfolio” and $u_2$ is a smaller, orthogonal one.


In [ ]:
u1 = vecs[:, 0]
u2 = vecs[:, 1]

# We care about directions, so give both eigenvector arrows
# the same visual length based on the larger eigenvalue.
length = 1.1 * np.sqrt(vals[0])
scale1 = length * 1.25
scale2 = length * 0.5

fig, ax = plt.subplots(figsize=(9, 6))

ax.plot(ellipse[:, 0], ellipse[:, 1], linewidth=2, label="1-sigma ellipse")
ax.scatter(Xc[:, 0], Xc[:, 1], label="Centered states")

ax.arrow(
    0,
    0,
    u1[0] * scale1,
    u1[1] * scale1,
    length_includes_head=True,
    head_width=0.008,
    head_length=0.02,
)
ax.arrow(
    0,
    0,
    u2[0] * scale2,
    u2[1] * scale2,
    length_includes_head=True,
    head_width=0.008,
    head_length=0.02,
)

ax.annotate(
    "$u_1$ (PC1)",
    xy=(u1[0] * scale1, u1[1] * scale1),
    textcoords="offset points",
    xytext=(10, -15),
    ha="left",
    va="top",
)
ax.annotate(
    "$u_2$ (PC2)",
    xy=(u2[0] * scale2, u2[1] * scale2),
    textcoords="offset points",
    xytext=(10, 10),
    ha="left",
    va="bottom",
)

ax.axhline(0, linewidth=1)
ax.axvline(0, linewidth=1)

ax.set_xlabel("Centered payoff $X_1-\mu_1$")
ax.set_ylabel("Centered payoff $X_2-\mu_2$")
ax.set_title(
    "Eigenvectors = principal axes; "
    "sqrt(eigenvalues) = axis lengths",
)
ax.grid(True, alpha=0.3)
ax.legend(loc="upper left")
ax.set_aspect("equal", adjustable="box")
ax.margins(0.15)

save_fig(fig, "fig_cov_eigenvectors.png")
plt.show()


## 3) Closed-form eigenvalues/eigenvectors for the 2×2 covariance matrix

Write:

$$
\Sigma=\begin{pmatrix} a & c\\ c & b\end{pmatrix},
\quad a=\sigma_1^2,\; b=\sigma_2^2,\; c=\sigma_{12}.
$$

Eigenvalues:

$$
\lambda_{\pm} = \frac{(a+b)\pm\sqrt{(a-b)^2+4c^2}}{2}.
$$

A corresponding eigenvector can be taken (up to scale) as:

$$
u(\lambda)\propto \begin{pmatrix} c\\ \lambda-a\end{pmatrix}\quad (c\neq 0).
$$

Eigenvectors are defined only up to sign: $u$ and $-u$ represent the same direction.


In [ ]:
a, b, c = Sigma[0, 0], Sigma[1, 1], Sigma[0, 1]
disc = np.sqrt((a - b)**2 + 4*c**2)

lam_plus  = 0.5*((a + b) + disc)
lam_minus = 0.5*((a + b) - disc)

print(f"Analytic eigenvalues: {lam_plus:.6f} {lam_minus:.6f}")
print("NumPy eigenvalues    :", vals)

u_plus  = unit(np.array([c, lam_plus  - a])) if abs(c) > 1e-14 else unit(np.array([1.0, 0.0]))
u_minus = unit(np.array([c, lam_minus - a])) if abs(c) > 1e-14 else unit(np.array([0.0, 1.0]))

err_plus = np.linalg.norm(Sigma @ u_plus  - lam_plus * u_plus)
err_minus = np.linalg.norm(Sigma @ u_minus - lam_minus * u_minus)

print("\nCheck Σu=λu (norm errors):")
print(f"  +: {err_plus:.3e}")
print(f"  -: {err_minus:.3e}")

align_plus = abs(np.dot(u_plus,  vecs[:, 0]))
align_minus = abs(np.dot(u_minus, vecs[:, 1]))

print("\nDirection alignment with NumPy eigenvectors (abs dot):")
print(f"  + vs u1: {align_plus:.6f}")
print(f"  - vs u2: {align_minus:.6f}")


## 4) Positive semidefinite (PSD) vs positive definite (PD)

For symmetric matrices:

- **PSD:** $z^\top\Sigma z \ge 0$ for all $z$.
- **PD:**  $z^\top\Sigma z > 0$ for all nonzero $z$.

Here $z^\top\Sigma z$ is the variance of the scalar random variable $z^\top X$. Covariance matrices are PSD because $z^\top\Sigma z=\mathbb{V}[z^\top X]\ge 0$ by definition.

- **PSD intuition:** every portfolio variance is nonnegative, but some directions $z$ may have zero variance (perfect linear dependence).
- **PD intuition:** every nontrivial direction carries strictly positive risk; there are no exact linear redundancies.

### 2×2 PSD condition
For $\Sigma=\begin{pmatrix}a&c\\c&b\end{pmatrix}$:

$$
\Sigma\succeq 0 \iff a\ge 0,\; b\ge 0,\; ab-c^2\ge 0.
$$

Equivalently, all eigenvalues are nonnegative.

The determinant
$$
\det(\Sigma)=ab-c^2
$$
measures how “2D” the risk is: $\det=0$ implies singular (rank 1) covariance, so all uncertainty lives on a single line in payoff space.


In [ ]:
det = np.linalg.det(Sigma)
print(f"det(Sigma) = {det:.6f}")
print("Eigenvalues =", vals)
print("PSD (all eigenvalues >= 0) =", np.all(vals >= -1e-12))
print("PD  (all eigenvalues  > 0) =", np.all(vals >  1e-12))


### 4.1 Visual PSD test: sample random directions

Compute $q(z)=z^\top\Sigma z$ for many unit vectors $z$.  
For PSD matrices, $q(z)$ never goes negative.


In [ ]:
rng = np.random.default_rng(42)

Z = rng.normal(size=(20000, 2))
Z = Z / np.linalg.norm(Z, axis=1, keepdims=True)
q = np.einsum("ni,ij,nj->n", Z, Sigma, Z)

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(q, bins=60)
ax.set_title("PSD check: histogram of $z^T\Sigma z$ over random unit vectors $z$")
ax.set_xlabel("$z^T\Sigma z$")
ax.set_ylabel("count")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_psd_hist.png")
plt.show()

print(f"min q = {q.min():.6f}  max q = {q.max():.6f}")


### 4.2 A non-PSD “covariance matrix” (what breaks)

If you inflate the off-diagonal $c$ beyond the PSD bound $|c|\le \sqrt{ab}$, one eigenvalue becomes negative.
Then some directions $z$ have $z^\top\Sigma z<0$, which is impossible for a true covariance matrix.


In [ ]:
c_bad = 1.5*np.sqrt(a*b)  # violates |c| <= sqrt(ab)
Sigma_bad = np.array([[a, c_bad], [c_bad, b]])

vals_bad, vecs_bad = eig_sorted(Sigma_bad)
print("Sigma_bad =\n", Sigma_bad)
det_bad = np.linalg.det(Sigma_bad)
print(f"det(Sigma_bad) = {det_bad:.6f}")
print("eigenvalues =", vals_bad)

Z = rng.normal(size=(20000, 2))
Z = Z / np.linalg.norm(Z, axis=1, keepdims=True)
q_bad = np.einsum("ni,ij,nj->n", Z, Sigma_bad, Z)

fig, ax = plt.subplots(figsize=(7, 4))
ax.hist(q_bad, bins=60)
ax.set_title("Non-PSD example: $z^T\Sigma z$ can be negative")
ax.set_xlabel("$z^T\Sigma z$")
ax.set_ylabel("count")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_nonpsd_hist.png")
plt.show()

share_neg = (q_bad < 0).mean()
print(f"min q_bad = {q_bad.min():.6f}  share negative = {share_neg:.6f}")


## 5) Portfolio variance in the two-asset case

Let weights $w=(w_1,w_2)$ and portfolio payoff $X_p=w_1X_1+w_2X_2$. Then:

$$
\mathbb{V}[X_p]=w^\top\Sigma w.
$$

This formula is just the geometric story in portfolio language: we pick a direction $w$ in payoff space, and $w^\top\Sigma w$ gives the variance of the projection of $X$ onto that direction.

With budget constraint $w_1+w_2=1$ (so $w_2=1-w_1$), this becomes a quadratic function of $w_1$ that you can visualize as a parabola over the weight axis.

### Minimum variance weight (only $w_1+w_2=1$)
$$
w_1^\star=\frac{\sigma_2^2-\sigma_{12}}{\sigma_1^2+\sigma_2^2-2\sigma_{12}},\quad w_2^\star=1-w_1^\star.
$$

At this point, the directional variance $w^\top\Sigma w$ is minimized; in the geometric picture this corresponds to **tilting the weight vector** so that the 1-sigma ellipse is as “narrow” as possible along the portfolio direction.


In [ ]:
w1_star = (b - c) / (a + b - 2*c)
w2_star = 1 - w1_star
var_star = w1_star**2*a + w2_star**2*b + 2*w1_star*w2_star*c

w1_min = min(-0.5, w1_star - 0.2)
w1_max = max(1.5, w1_star + 0.2)
w1 = np.linspace(w1_min, w1_max, 501)
w2 = 1 - w1

var_p = (w1**2)*a + (w2**2)*b + 2*w1*w2*c

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(w1, var_p, label="portfolio variance")
ax.vlines(
    w1_star,
    0.0,
    var_star,
    colors="tab:red",
    linewidth=1.5,
    linestyles="--",
)
ax.scatter(
    [w1_star],
    [var_star],
    color="tab:red",
    zorder=3,
    label="min-variance portfolio $w_1^*$",
)

ax.set_title("Portfolio variance $w^T\Sigma w$ vs weight $w_1$ with $w_2=1-w_1$")
ax.set_xlabel("$w_1$")
ax.set_ylabel("Variance")
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(fig, "fig_cov_portfolio_variance.png")
plt.show()

print(f"w* = ({w1_star:.6f}, {w2_star:.6f})")
print(f"var(w*) = {var_star:.6f}")


### 5.1 Eigenvalues via the Rayleigh quotient (risk extremes)

If instead you constrain $\|w\|_2=1$ (unit leverage), then:

$$
\lambda_{\min}\le \frac{w^\top\Sigma w}{w^\top w}\le \lambda_{\max}.
$$

The maximizing $w$ is the top eigenvector $u_1$; the minimizing $w$ is $u_2$.


In [ ]:
W = rng.normal(size=(30000, 2))
rq = np.einsum("ni,ij,nj->n", W, Sigma, W) / np.einsum("ni,ni->n", W, W)

rq_min = rq.min()
rq_max = rq.max()
lam_min = vals.min()
lam_max = vals.max()

print(
    "Rayleigh quotient min/max (empirical): "
    f"{rq_min:.6f} {rq_max:.6f}",
)
print(
    "Eigenvalue min/max: "
    f"{lam_min:.6f} {lam_max:.6f}",
)


## 6) Diagonalization: rotate into the eigenbasis (principal components)

Because $\Sigma$ is symmetric:

$$
\Sigma = Q\Lambda Q^\top
$$

with orthonormal $Q=[u_1\;u_2]$ and diagonal $\Lambda=\mathrm{diag}(\lambda_1,\lambda_2)$.

If you rotate centered payoffs $Y = X_c Q$, then:

$$
\mathrm{Cov}(Y)=\Lambda.
$$

So in the eigenbasis, the covariance matrix becomes diagonal: **uncorrelated principal components**. Each component is a factor portfolio with variance equal to one eigenvalue, and the 1-sigma ellipse becomes an axis-aligned ellipse (in fact, a circle after rescaling).

You can think of this rotation as **finding a coordinate system where the payoff cloud is as simple as possible**: all co-movement is pushed into separate, orthogonal axes. This is exactly the viewpoint of principal component analysis (PCA).


In [ ]:
Q = vecs
Lam = np.diag(vals)

err = np.linalg.norm(Sigma - Q @ Lam @ Q.T)
print(f"||Sigma - QΛQ^T|| = {err:.3e}")
print("Q^T Sigma Q =\n", Q.T @ Sigma @ Q)

Y = Xc @ Q
_, Sigma_pc = cov_from_states(Y, p)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(Y[:, 0], Y[:, 1])

for i, s in enumerate(states):
    ax.annotate(s, (Y[i, 0], Y[i, 1]), textcoords="offset points", xytext=(6, 6))

ax.axhline(0, linewidth=1)
ax.axvline(0, linewidth=1)
ax.set_xlabel("PC1 coordinate")
ax.set_ylabel("PC2 coordinate")
ax.set_title("Centered states expressed in the eigenbasis (principal components)")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_pca_scatter.png")
plt.show()

print("Covariance in PC coordinates (prob-weighted) =\n", Sigma_pc)


## 7) Correlation matrix and scale effects

Covariance depends on units. Correlation is scale-free:

$$
\rho_{12}=\frac{\sigma_{12}}{\sigma_1\sigma_2}.
$$

A standardized covariance matrix is the correlation matrix:

$$
R = D^{-1/2}\Sigma D^{-1/2},\quad D=\mathrm{diag}(\sigma_1^2,\sigma_2^2).
$$

Eigenvalues/eigenvectors of $R$ describe co-movement structure independent of scale.


In [ ]:
D_inv_sqrt = np.diag(1 / np.sqrt(np.diag(Sigma)))
R = D_inv_sqrt @ Sigma @ D_inv_sqrt

vals_R, vecs_R = eig_sorted(R)

print("R =\n", R)
print("eig(R) =", vals_R)
print("eigvecs(R) (cols) =\n", vecs_R)


## 8) Estimation: sample covariance convergence (simulation)

In practice, $\Sigma$ is unknown and must be estimated from data (for example, a time series of returns). With small samples it is noisy and may be nearly singular.

In this section we simulate a bivariate normal with target covariance $\Sigma$ and track how the sample covariance converges as we increase the number of observations. You can read the plots as **slowly sharpening versions of the same 1-sigma ellipse**: early on, the sampled cloud is jagged and the estimated ellipse wiggles; later, both settle near the true shape.


In [ ]:
# Build A such that Sigma = A A^T using eigen-decomposition
vals, vecs = eig_sorted(Sigma)
A = vecs @ np.diag(np.sqrt(np.maximum(vals, 0)))

rng = np.random.default_rng(0)

def sample_cov(n):
    Z = rng.normal(size=(n, 2))
    Xsim = Z @ A.T  # mean 0, cov approx Sigma
    S = np.cov(Xsim.T, bias=False)  # sample covariance (n-1 denom)
    return S

ns = np.array([10, 20, 50, 100, 200, 500, 1000, 3000, 10000])
errs = []
for n in ns:
    S = sample_cov(int(n))
    errs.append(np.linalg.norm(S - Sigma))

errs = np.array(errs)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(ns, errs, marker="o")
ax.set_xscale("log")
ax.set_title("Convergence of sample covariance to target Σ (log x-axis)")
ax.set_xlabel("sample size n")
ax.set_ylabel("||S_n - Σ||")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_sample_convergence.png")
plt.show()

print("errors:", errs)


## 9) Near-singularity and rank deficiency

A 2×2 covariance matrix becomes singular exactly when:

$$
\det(\Sigma)=ab-c^2=0\quad\Longleftrightarrow\quad\lvert\rho_{12}\rvert=1.
$$

Then one eigenvalue is 0: the uncertainty is effectively one-dimensional (a “one-factor” structure). Geometrically, the 1-sigma ellipse collapses to a line segment; economically, one payoff is an exact linear combination of the other.

This is the limiting case of highly correlated assets in practice: as $\lvert\rho_{12}\rvert$ approaches 1, the ellipse becomes more and more elongated and numerical algorithms start to struggle, because many directions $w$ have almost the same variance.


In [ ]:
sigma1 = 0.20
sigma2 = 0.15
rho_target = 0.999
c_near = rho_target * sigma1 * sigma2

Sigma_near = np.array([[sigma1**2, c_near], [c_near, sigma2**2]])
vals_near, vecs_near = eig_sorted(Sigma_near)

print("Sigma_near =\n", Sigma_near)
det_near = np.linalg.det(Sigma_near)
print(f"det(Sigma_near) = {det_near:.6f}")
print("eigenvalues =", vals_near)

# visualize ellipse (very elongated)
theta = np.linspace(0, 2*np.pi, 400)
circle = np.column_stack([np.cos(theta), np.sin(theta)])
sqrtSigma_near = vecs_near @ np.diag(np.sqrt(np.maximum(vals_near, 0))) @ vecs_near.T
ellipse_near = circle @ sqrtSigma_near.T

fig, ax = plt.subplots(figsize=(7, 5))
ax.plot(ellipse_near[:, 0], ellipse_near[:, 1], linewidth=2)
ax.axhline(0, linewidth=1)
ax.axvline(0, linewidth=1)
ax.set_title("Near-singular covariance: ellipse becomes extremely elongated")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid(True, alpha=0.3)
save_fig(fig, "fig_cov_near_singular.png")
plt.show()


## 10) State prices, SDF, and covariance

In a one-period world, no-arbitrage pricing can be written in terms of a **stochastic discount factor (SDF)** $m(\omega)$ or, equivalently, **state prices** $q(\omega)$. For any asset with payoff $X(\omega)$ and current price $P$, the pricing equation is

$$
P = \mathbb{E}[m X] = \sum_{\omega} p(\omega)\, m(\omega) X(\omega).
$$

If we work with gross returns $R(\omega)=X(\omega)/P$ and take $P=1$, this becomes

$$
1 = \mathbb{E}[m R].
$$

Intuitively, $m(\omega)$ is high in **bad states** (where marginal utility of wealth is high) and low in **good states**.

In this section we construct a simple SDF for our two-asset world, chosen to be larger when the equal-weight ``market" portfolio does poorly, and visualize how its **covariance with each asset** relates to expected excess returns.


In [ ]:
# Simple SDF construction tied to the equal-weight market portfolio
Rf = 1.02  # risk-free gross return (2% per period)

w_market = np.array([0.5, 0.5])
R_market = X @ w_market
mu_market = (p * R_market).sum()

# Stylized SDF: high when the market does badly, low when it does well
m_raw = 1.0 / R_market
k = 1.0 / (Rf * (p * m_raw).sum())
m = k * m_raw

# Check pricing of risk-free and risky assets via E[m R] ≈ 1
price_rf = (p * m * Rf).sum()
price_1 = (p * m * X1).sum()
price_2 = (p * m * X2).sum()

print(f"Price of risk-free (target 1.000000): {price_rf:.6f}")
print(f"Price of asset 1    (target 1.000000): {price_1:.6f}")
print(f"Price of asset 2    (target 1.000000): {price_2:.6f}")

# Covariance of SDF with each asset return
m_bar = (p * m).sum()
cov_m_R1 = (p * (m - m_bar) * (X1 - mu[0])).sum()
cov_m_R2 = (p * (m - m_bar) * (X2 - mu[1])).sum()

print(f"Cov(m, R1) = {cov_m_R1:.6f}")
print(f"Cov(m, R2) = {cov_m_R2:.6f}")

# Visualize payoffs and SDF across states (sorted by market return)
idx = np.argsort(R_market)
states_sorted = states[idx]
R_market_sorted = R_market[idx]
m_sorted = m[idx]

x = np.arange(len(states_sorted))

fig, ax1 = plt.subplots(figsize=(8, 4))
# Bars centered at integer positions for the market payoff
bars = ax1.bar(x, R_market_sorted, width=0.6, label="Market payoff R_m")
ax1.set_ylabel("Gross return")
ax1.set_xticks(x)
ax1.set_xticklabels(states_sorted, rotation=30, ha="right")
y_min = 0.0
y_max = R_market_sorted.max() * 1.25
ax1.set_ylim(y_min, y_max)

ax2 = ax1.twinx()
ax2.plot(
    x,
    m_sorted,
    color="tab:orange",
    marker="o",
    label="SDF m(ω)",
)
ax2.set_ylabel("SDF level")
m_min = m_sorted.min() * 0.95
m_max = m_sorted.max() * 1.05
ax2.set_ylim(m_min, m_max)
ax2.grid(False)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(
    lines1 + lines2,
    labels1 + labels2,
    loc="upper center",
)

ax1.set_title("Stylized SDF: high in bad states, low in good states")
fig.tight_layout()
save_fig(fig, "fig_cov_sdf_states.png")
plt.show()


## 11) Mean–variance pricing and a CAPM-style view

In mean–variance theory we approximate investors by their preferences over expected return and return variance. With a risk-free gross return $R_f$ and a risky market portfolio $R_m$, the Capital Asset Pricing Model (CAPM) states

$$
\mathbb{E}[R_i] - R_f = \beta_i \bigl(\mathbb{E}[R_m] - R_f\bigr),\qquad \beta_i = \frac{\mathrm{Cov}(R_i,R_m)}{\mathrm{Var}(R_m)}.
$$

We now treat an equal-weight portfolio of our two assets as the market, generate many random portfolios, and visualize both the **mean–variance cloud** and the **Security Market Line** (expected excess return versus beta).


In [ ]:
# Equal-weight market portfolio and its moments
Rf = 1.02
w_market = np.array([0.5, 0.5])
R_market = X @ w_market
mu_market = (p * R_market).sum()
var_market = (p * (R_market - mu_market)**2).sum()

# Random portfolios on the two-asset span
rng = np.random.default_rng(123)
W = rng.normal(size=(300, 2))
W = W / W.sum(axis=1, keepdims=True)

mu_port = W @ mu
var_port = np.einsum("ni,ij,nj->n", W, Sigma, W)
sigma_port = np.sqrt(var_port)

# Mean–variance picture (σ on x-axis, E[R] on y-axis)
fig, ax = plt.subplots(figsize=(7, 5))
asset_sigma = np.sqrt(np.diag(Sigma))
ax.scatter(sigma_port, mu_port, alpha=0.4, s=15, label="random portfolios")
ax.scatter(asset_sigma, mu, color='yellow', label="individual assets")
ax.scatter(
    np.sqrt(var_market),
    mu_market,
    color="tab:red",
    label="market (50/50)",
)

ax.set_xlabel("Standard deviation")
ax.set_ylabel("Expected gross return")
ax.set_title("Mean–variance picture for the two-asset world")
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(fig, "fig_cov_mean_variance.png")
plt.show()

In [ ]:
# Security Market Line: expected excess return vs beta
beta_assets = (Sigma @ w_market) / var_market
beta1, beta2 = beta_assets

beta_port = (W @ (Sigma @ w_market)) / var_market
excess_mu_assets = mu - Rf
excess_mu_port = mu_port - Rf
excess_mu_market = mu_market - Rf

beta_grid = np.linspace(beta_port.min() - 0.1, beta_port.max() + 0.1, 50)
excess_sml = beta_grid * (mu_market - Rf)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(beta_port, excess_mu_port, alpha=0.4, s=15, label="random portfolios")
ax.scatter(beta_assets, excess_mu_assets, color="yellow", label="assets")
ax.scatter(
    [1.0],
    [excess_mu_market],
    color="tab:red",
    label="market (beta=1)",
)
ax.plot(beta_grid, excess_sml, color="blue", alpha=0.8, linewidth=2, label="Security Market Line")

ax.set_xlabel("Beta relative to market")
ax.set_ylabel("Expected excess return E[R] - R_f")
ax.set_title("CAPM-style Security Market Line from two-asset world")
ax.grid(True, alpha=0.3)
ax.legend()
save_fig(fig, "fig_cov_sml.png")
plt.show()

## 12) A two-asset factor model

Finally, we can view the equal-weight portfolio $R_m$ as a **single factor** and express each asset return as

$$
R_i = \alpha_i + \beta_i R_m + \varepsilon_i,\qquad \mathbb{E}[\varepsilon_i]=0,\ \mathrm{Cov}(\varepsilon_i,R_m)=0.
$$

This is the simplest form of a one-factor model. It shows how covariance with the factor (through $\beta_i$) drives expected returns while idiosyncratic noise $\varepsilon_i$ averages out in portfolios.

We estimate $\alpha_i$ and $\beta_i$ in the discrete state world and visualize the regression lines.


In [ ]:
Rf = 1.02
w_market = np.array([0.5, 0.5])
R_market = X @ w_market
mu_market = (p * R_market).sum()
var_market = (p * (R_market - mu_market)**2).sum()

def factor_reg(R, name):
    """Weighted one-factor regression on the market portfolio."""
    mu_R = (p * R).sum()
    cov_RM = (p * (R - mu_R) * (R_market - mu_market)).sum()
    beta = cov_RM / var_market
    alpha = mu_R - beta * mu_market
    eps = R - (alpha + beta * R_market)
    mu_eps = (p * eps).sum()
    var_eps = (p * (eps - mu_eps)**2).sum()
    print(
        f"{name}: alpha = {alpha:.6f}, beta = {beta:.6f}, "
        f"Var(eps) = {var_eps:.6f}",
    )
    return alpha, beta, eps

alpha1, beta1, eps1 = factor_reg(X1, "Asset 1")
alpha2, beta2, eps2 = factor_reg(X2, "Asset 2")

fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharex=True, sharey=True)
for ax, R, alpha, beta, name in [
    (axes[0], X1, alpha1, beta1, "Asset 1"),
    (axes[1], X2, alpha2, beta2, "Asset 2"),
]:
    ax.scatter(R_market, R)
    ax.set_title(name)
    ax.set_xlabel("Market return R_m")
    ax.grid(True, alpha=0.3)

    Rm_line = np.linspace(R_market.min(), R_market.max(), 50)
    R_line = alpha + beta * Rm_line
    ax.plot(Rm_line, R_line, color="tab:orange", linewidth=2)

axes[0].set_ylabel("Asset return R_i")
plt.suptitle("One-factor view: assets as alpha + beta × market + noise")
plt.tight_layout()
save_fig(fig, "fig_cov_factor_regression.png")
plt.show()


## 13) Quick reference (2 assets)

Given

$$
\Sigma=\begin{pmatrix}\sigma_1^2 & \sigma_{12}\\ \sigma_{12} & \sigma_2^2\end{pmatrix},
$$

- PSD iff $\sigma_1^2\ge 0$, $\sigma_2^2\ge 0$, $\sigma_{12}^2\le \sigma_1^2\sigma_2^2$.
- Eigenvalues:
  $$
  \lambda_{\pm}=\frac{(\sigma_1^2+\sigma_2^2)\pm\sqrt{(\sigma_1^2-\sigma_2^2)^2+4\sigma_{12}^2}}{2}.
  $$
- Eigenvectors = principal risk directions; eigenvalues = variances along them.
- Portfolio variance: $w^\top\Sigma w$.
- $\det(\Sigma)=\sigma_1^2\sigma_2^2-\sigma_{12}^2$ indicates singularity and effective rank.



<img src="https://hilpisch.com/tpq_logo_bic.png" width="20%" align="right">
